In [1]:
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
import pickle
import time
import random
from model.gpt import *
# code starts here

device = "cuda"
model = GPT(GPTConfig(vocab_size=50304)).to(device)

torch.set_float32_matmul_precision('high')

pretrained = torch.load('finetuned.pth')

model.load_state_dict(pretrained['model'])

model.config = pretrained['config']

start_epoch = pretrained['epoch']
start_step = pretrained['step']

print('loading data')

with open('qa_pairs.pkl', 'rb') as f:
    data = pickle.load(f)

train_data = data[:int(len(data) * 0.999)]
val_data = data[int(len(data) * 0.999):]



loading data


In [3]:
model.eval()
with torch.no_grad():
    val_loss_accum = 0
    val_loss_steps = len(val_data)
    for i in range(val_loss_steps):
        xy = val_data[i]
        x, y = torch.tensor(xy[:-1]), torch.tensor(xy[1:])
        x, y = x.unsqueeze(0).to(device), y.unsqueeze(0).to(device)
        with torch.autocast(device_type=device, dtype=torch.bfloat16):
            logits, loss = model(x, y)
        loss /= val_loss_steps
        val_loss_accum += loss.detach()
print(f"Validation loss = {val_loss_accum.item()}")

Validation loss = 2.3874025344848633
